<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 99% of this material has been adapted from [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<!--NAVIGATION-->
 < [Array](8_Array.ipynb) | [Contents](index.ipynb) |  [Asset Management](10_Import.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/9_SpecializedAlgorithms.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<center>
<h1>Google Earth Engine with Python </h1>
<h2> Specialized Algorithms </h2>
</center>
<h2> Topics:</h2>

1. Supervised Classification
2. Unsupervised Classification (clustering)
3. Landsat Algorithms
4. Sentinel-1 Algorithms
5. Resampling and Reducing Resolution


## Connecting GEE with Google Services

- **Authenticate to Earth Engine**

In [0]:
!pip install earthengine-api #earth-engine Python API

In [0]:
!earthengine authenticate 

- **Authenticate to Google Drive (OPTIONAL)**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

- **Authenticate to Google Cloud (OPTIONAL)**

In [0]:
from google.colab import auth
auth.authenticate_user()

## Testing the software setup

In [0]:
# Earth Engine Python API
import ee
ee.Initialize()

In [0]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

In [0]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1. Supervised Classification

The `Classifier` package handles supervised classification in Earth Engine. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.

2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be small, consecutive, non-negative integers representing classes. If necessary, use `remap()` to convert class values to consecutive integers starting from 0. The predictors should be numeric.


Training and/or validation data can come from a variety of sources. To collect training data interactively in Earth Engine, you can use the geometry drawing tools (see the [Code Editor section](https://developers.google.com/earth-engine/playground)). Alternatively, you can import predefined training data from an Earth Engine table asset or a Fusion Table (see the [Importing section](https://developers.google.com/earth-engine/importing) for details). Get a classifier from one of the constructors in `ee.Classifier`. Train the classifier using `classifier.train()`. Classify an `Image` or `FeatureCollection` using `classify()`. The following example uses a Classification and Regression Trees (CART) classifier ([Breiman et al. 1984](https://books.google.com.pe/books?id=JwQx-WOmSyQC&redir_esc=y)) to predict forest and non-forest areas in the Amazon:

In [0]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11'];

# Load a Landsat 8 image to be used for prediction.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_232067_20130726')\
          .select(bands)

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('ft:10X7SUjDTiFJDyIA58zLcptK8pwBwjj1BV12SQOgJ')\
           .remap([1, 2], [0, 1], 'class')

# Overlay the points on the imagery to get training.
training = image.sampleRegions(
  collection=points,
  properties=['class'],
  scale=30)


# Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, 'class', bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

# Display the inputs and the results.
center = points.geometry().centroid().getInfo()['coordinates']
center.reverse()

image_vizparam = {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}
image_token = image.getMapId(image_vizparam)

classify_vizparam = {'min': 0, 'max': 1, 'palette': ['00FF00', 'FF0000']}
classify_token = classified.getMapId(classify_vizparam)

Mapdisplay(center = center,
           dicc = {'LANDSAT':image_token,
                   'FOREST':classify_token},
           zoom_start=8)

In this example, the [training points in the Fusion Table](https://fusiontables.google.com/DataSource?docid=10X7SUjDTiFJDyIA58zLcptK8pwBwjj1BV12SQOgJ#rows:id=1) store only the class label. Note that `remap()` is used to convert the training property to consecutive integers starting at 0. Also note the use of `image.sampleRegions()` to get the predictors into the table and create a training dataset. To train the classifier, specify the name of the class label property and a list of properties in the training table which the classifier should use for predictors. The number and order of the bands in the image to be classified must exactly match the order of the properties list provided to `classifier.train()`. Use `image.select()` to ensure that the classifier schema matches the image.

If the training data are polygons representing homogenous regions, every pixel in each polygon is a training point. Use polygons to train as illustrated in the following example:

In [0]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Load an image over a portion of southern California, USA.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_040037_20131114')\
          .select(bands)

# Load training polygons from a Fusion Table.
# The 'class' property stores known class labels.
polygons = ee.FeatureCollection('ft:1vYn7-uO80vAVpZxi81yXzx55jgtztsZaLxG5TWYH')

# Get the values for all pixels in each polygon in the training.
training = image.sampleRegions(
  collection=polygons,
  properties=['class'],
  scale=30)

# Create an SVM classifier with custom parameters.
classifier = ee.Classifier.svm(
  kernelType='RBF',
  gamma=0.5,
  cost=10)

# Train the classifier.
trained = classifier.train(training, 'class', bands)

# Classify the image.
classified = image.classify(trained)

# Create a palette to display the classes.
palette =['006400', '32CD32', 'EEE8AA',
          '8B4513', '98FB98', '00FA9A',
          '90EE90', '00008B', 'FF8C00',
          'ADFF2F', '808080']

# Display the classification result and the input image.
center = [33.30, -117.3]
token_image = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.5, 'gamma': 2})
token_classified = classified.getMapId({'min': 0, 'max': 10, 'palette': palette})
Mapdisplay(center,{'LANDSAT':token_image,'Vegetation_Type':token_classified},zoom_start=9)

This example uses a Support Vector Machine (SVM) classifier ([Burges 1998](https://rd.springer.com/article/10.1023%2FA%3A1009715923555)). Note that the SVM is specified with a set of custom parameters. Without a priori information about the physical nature of the prediction problem, optimal parameters are unknown. See [Hsu et al. (2003)](https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf) for a rough guide to choosing parameters for an SVM.

To assess the accuracy of a classifier, use a `ConfusionMatrix` ([Stehman 1997](https://www.sciencedirect.com/science/article/abs/pii/S0034425797000837#aep-article-footnote-id1)). The following example uses `sample()` to generate training and validation data from a MODIS reference image and compares confusion matrices representing training and validation accuracy:

In [0]:
# Define a region of interest as a point.  Change the coordinates
# to get a classification of any place where there is imagery.
roi = ee.Geometry.Point(-122.3942, 37.7295)

# Load Landsat 5 input imagery.
# 1. Filter to get only one year of images.
# 2. Filter to get only images under the region of interest.
# 3. Sort by scene cloudiness, ascending.
# 4. Get the first (least cloudy) scene.
landsat = ee.Image(ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
                     .filterDate('2011-01-01', '2011-12-31')\
                     .filterBounds(roi)\
                     .sort('CLOUD_COVER')\
                     .first())

# Compute cloud score.
cloudScore = ee.Algorithms.Landsat.simpleCloudScore(landsat).select('cloud')

# Mask the input for clouds.  Compute the min of the input mask to mask
# pixels where any band is masked.  Combine that with the cloud mask.
Input = landsat.updateMask(landsat.mask().reduce('min').And(cloudScore.lte(50)))

# Use MODIS land cover, IGBP classification, for training.
modis = ee.Image('MODIS/051/MCD12Q1/2011_01_01')\
          .select('Land_Cover_Type_1')

# Sample the input imagery to get a FeatureCollection of training data.
training = Input.addBands(modis).sample(numPixels=5000, seed=0)

# Make a Random Forest classifier and train it.
classifier = ee.Classifier.randomForest(10)\
               .train(training, 'Land_Cover_Type_1')

# Classify the input imagery.
classified = Input.classify(classifier)

# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

# Sample the input with a different random seed to get validation data.
validation = Input.addBands(modis)\
                  .sample(numPixels=5000,seed=1)\
                  .filter(ee.Filter.neq('B1', None)) # Filter the result to get rid of any null pixels.

# Classify the validation data.
validated = validation.classify(classifier)

# Get a confusion matrix representing expected accuracy.
testAccuracy = validated.errorMatrix('Land_Cover_Type_1', 'classification')
print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

# Define a palette for the IGBP classification.
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

# Display the input and the classification.
Input_token = Input.getMapId({'bands': ['B3', 'B2', 'B1'], 'max': 0.4})
classified_token = classified.getMapId({'palette': igbpPalette, 'min': 0, 'max': 17})
center = [37.7295,-122.3942]
Mapdisplay(center, {'Landsat':Input_token,'LandUse':classified_token}, zoom_start=10)

Resubstitution error matrix:  [[1763, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 4, 0, 0, 0], [5, 197, 0, 0, 0, 2, 0, 0, 13, 3, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 13, 0, 0, 0, 68, 0, 0, 9, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 16, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 42, 8, 1, 2, 0, 0, 0, 0, 0, 0], [1, 10, 0, 0, 0, 2, 0, 0, 679, 7, 3, 0, 14, 3, 0, 0, 0], [1, 2, 0, 0, 0, 5, 0, 0, 30, 237, 5, 0, 5, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 2, 23, 11, 276, 0, 9, 3, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 16, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 17, 5, 11, 0, 574, 4, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 2, 3, 2, 0, 16, 343, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 1, 0, 4, 0, 32, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6]]
Training overall accuracy:  0.93693495

This example uses a random forest ([Breiman 2001](https://rd.springer.com/article/10.1023/A:1010933404324)) classifier with 10 trees to downscale MODIS data to Landsat resolution. The `sample()` method generates two random samples from the MODIS data: one for training and one for validation. The training sample is used to train the classifier. You can get resubstitution accuracy on the training data from `classifier.confusionMatrix()`. To get validation accuracy, classify the validation data. This adds a `classification` property to the validation `FeatureCollection`. Call `errorMatrix()` on the classified `FeatureCollection` to get a confusion matrix representing validation (expected) accuracy.

Inspect the output to see that the overall accuracy estimated from the training data is much higher than the validation data. The accuracy estimated from training data is an overestimate because the random forest is “fit” to the training data. The expected accuracy on unknown data is lower, as indicated by the estimate from the validation data.

# 2. Unsupervised Classification (clustering)

The `ee.Clusterer` package handles unsupervised classification (or clustering) in Earth Engine. These algorithms are currently based on the algorithms with the same name in [Weka](http://www.cs.waikato.ac.nz/ml/weka/). 

Clusterers are used in the same manner as classifiers in Earth Engine. The general workflow for clustering is:

1. Assemble features with numeric properties in which to find clusters.
2. Instantiate a clusterer. Set its parameters if necessary.
3. Train the clusterer using the training data.
4. Apply the clusterer to an image or feature collection.
5. Label the clusters.

The training data is a `FeatureCollection` with properties that will be input to the clusterer. Unlike classifiers, there is no input class value for an `Clusterer`. Like classifiers, the data for the train and apply steps are expected to have the same number of values. When a trained clusterer is applied to an image or table, it assigns an integer cluster ID to each pixel or feature.

Here is a simple example of building and using an `ee.Clusterer`:

In [0]:
# Load a pre-computed Landsat composite for input.
Input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

# Define a region in which to generate a sample of the input.
region = ee.Geometry.Rectangle(29.7, 30, 32.5, 31.7)
region_img = ee.Image().paint(region, 0, 2)
# Display the sample region.
center = [31.0,31.5]
Mapdisplay(center,{'Region':region_img.getMapId()},zoom_start=8)

In [0]:
# Make the training dataset.
training = Input.sample(
  region=region,
  scale=30,
  numPixels=5000
)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = Input.cluster(clusterer)
result_tk = result.randomVisualizer().getMapId()
# Display the clusters with random colors.
Mapdisplay(center, {'results':result_tk},zoom_start=8)

Please note:

- The same inputs should always produce the same outputs, but reordering the inputs can change the results.

- Training with as few as 10 bands * 100k points can produce an Out Of Memory error.
- Cobweb can can take a long time to finish and can produce a large number of clusters.
- The output clusters and their IDs are dependent on the algorithm and inputs.

# 3. Landsat Algorithms

## Landsat collection structure

The USGS has moved to using a collection-based strategy for producing Landsat scenes. On May 1st, 2017, they stopped producing old-style (pre-Collection) scenes and now are only producing Collection 1 versions.

The USGS now produces data in 3 categories for each satellite:

- Tier 1 (T1) - Data that meets geometric and radiometric quality requirements
- Tier 2 (T2) - Data that doesn't meet the Tier 1 requirements
- Real Time (RT) - Data that hasn't yet been evaluated (it takes as much as a month).

In order to allow access to both the validated T1 data and the newest real-time data together, we've sorted the scenes into 3 collections (per each satellite) as follows:

ID	| Description
------------|-----------------
LANDSAT/LC08/C01/T1_RT	| Landsat 8, Collection 1, Tier 1 + Real Time
LANDSAT/LC08/C01/T1	| Landsat 8, Collection 1, Tier 1 only
LANDSAT/LC08/C01/T2	| Landsat 8, Collection 1, Tier 2 only

Newly acquired scenes are added to the T1_RT collection daily. Once an RT scene gets reprocessed and categorized as either T1 or T2, it will be removed from the T1_RT collection and the new version will be added to the appropriate collection(s). If your work is sensitive to removals or potentially mis-registered scenes, you might want to stick to the T1 collection, but in general, it's very uncommon that any misregistration is large enough to notice on newly acquired scenes.

Each of these collections contains the raw data (i.e., scaled, at-sensor radiance). In addition, for each collection that contains T1 data, we are also providing a TOA collection containing the top-of-atmosphere reflectance. (e.g.: LANDSAT/LC08/C01/T1_RT_TOA).

Finally, because pre-Collection 1 scenes are no longer being produced, we will not be able to update the "old-style" Landsat collections (e.g.: LANDSAT/LC8_L1T). We plan to keep these collections for a while even after we've obtained all of the C1 data, but they will eventually be phased out.

### Landsat processing methods

Earth Engine contains a variety of Landsat specific processing methods. Specifically, there are methods to compute at-sensor radiance, top-of-atmosphere (TOA) reflectance, surface reflectance (SR), cloud score and cloud-free composites.

#### At-sensor radiance and TOA reflectance

The ‘raw’ scenes in Earth Engine contain imagery with digital numbers (DNs) that represent scaled radiance. The conversion of DNs to at-sensor radiance is a linear transformation using coefficients stored in scene metadata ([Chander et al. 2009](https://www.sciencedirect.com/science/article/abs/pii/S0034425709000169)). The `ee.Algorithms.Landsat.calibratedRadiance()` method performs this conversion. Conversion to TOA (or at-sensor) reflectance is a linear transformation that accounts for solar elevation and seasonally variable Earth-Sun distance. The TOA conversion is handled by the `ee.Algorithms.Landsat.TOA()` method. The TOA method converts thermal bands to brightness temperature. See [Chander et al. (2009)](https://www.sciencedirect.com/science/article/abs/pii/S0034425709000169) (or [this USGS site](https://landsat.usgs.gov/using-usgs-landsat-8-product) for Landsat 8) for more information about computing TOA reflectance or brightness temperature. The following example shows conversion from raw data to radiance and TOA reflectance for a Landsat 8 image:

In [0]:
center = [37.37, -122.04]

# Load a raw Landsat scene and display it.
raw = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')
raw_tk = raw.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 6000, 'max': 12000})

# Convert the raw data to radiance.
radiance = ee.Algorithms.Landsat.calibratedRadiance(raw)
radiance_tk = radiance.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 90})

# Convert the raw data to top-of-atmosphere reflectance.
toa = ee.Algorithms.Landsat.TOA(raw)
toa_tk = toa.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.2})

#Display them!
Mapdisplay(center,{'radiance':radiance_tk,'raw':raw_tk,'TOA':toa_tk})

### Surface reflectance

Conversion to surface reflectance generally requires some sort of atmospheric compensation. USGS uses the LEDAPS software ([Schmidt et al. 2013](https://pubs.er.usgs.gov/publication/ofr20131057)) to convert raw Landsat data to surface reflectance. LEDAPS requires a variety of atmospheric inputs to constrain an atmospheric model. These include outputs of a [6S model](http://6s.ltdri.org/index.html) run, a DEM, total column ozone, National Centers for Environmental Prediction (NCEP) modeled surface pressure, temperature and water vapor ([Kalnay et al. 1996](https://journals.ametsoc.org/doi/abs/10.1175/1520-0477(1996)077%3C0437%3ATNYRP%3E2.0.CO%3B2)).

You can access pre-computed surface reflectance images directly from the SR collections. For example, to load a Landsat 7 surface reflectance image, use:

In [0]:
srImage = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_044034_19990707')

The surface reflectance datasets for Collection 1 Landsat 4 through 7 are:

In [0]:
surfaceReflectanceL4 = ee.ImageCollection('LANDSAT/LT04/C01/T1_SR')
surfaceReflectanceL5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
surfaceReflectanceL7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')

#### Simple cloud score

For scoring Landsat pixels by their relative cloudiness, Earth Engine provides a rudimentary cloud scoring algorithm in the `ee.Algorithms.Landsat.simpleCloudScore()` method. The following example uses the cloud scoring algorithm to mask clouds in a Landsat 8 image:

In [0]:
center = [37.37, -122.04]

# Load a cloudy Landsat scene and display it.
cloudy_scene = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140926')
cloudy_tk=cloudy_scene.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.4})

# Add a cloud score band.  It is automatically called 'cloud'.
scored = ee.Algorithms.Landsat.simpleCloudScore(cloudy_scene)
# Create a mask from the cloud score and combine it with the image mask.
mask = scored.select(['cloud']).lte(20)

# Apply the mask to the image and display the result.
masked = cloudy_scene.updateMask(mask)
masked_tk=masked.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.4})

# Load a Landsat 8 composite and set the SENSOR_ID property.
mosaic = ee.Image(ee.ImageCollection('LANDSAT/LC8_L1T_8DAY_TOA').first())\
           .set('SENSOR_ID', 'OLI_TIRS')

# Cloud score the mosaic and display the result.
scored_mosaic = ee.Algorithms.Landsat.simpleCloudScore(mosaic)
mosaic_tk=scored_mosaic.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.4})

Mapdisplay(center,{'TOA':cloudy_tk,'masked':masked_tk,'TOA mosaic':mosaic_tk})

If you run this example in the Code Editor, try toggling the visibility of the TOA layers to compare the difference between the masked and unmasked imagery. (See [this section](https://developers.google.com/earth-engine/playground#layer-manager) of the Code Editor docs for instructions on how to do that). Observe that the input to `simpleCloudScore()` is a single Landsat TOA scene. Also note that `simpleCloudScore()` adds a band called `‘cloud’` to the input image. The cloud band contains the cloud score from 0 (not cloudy) to 100 (most cloudy). The previous example uses an arbitrary threshold (20) on the cloud score to mask cloudy pixels. To apply this algorithm to an Earth Engine mosaic of Landsat scenes, set the `SENSOR_ID` property:

In [0]:
# Load a Landsat 8 composite and set the SENSOR_ID property.
mosaic = ee.Image(ee.ImageCollection('LANDSAT/LC8_L1T_8DAY_TOA').first())\
           .set('SENSOR_ID', 'OLI_TIRS')

# Cloud score the mosaic and display the result.
scored_mosaic = ee.Algorithms.Landsat.simpleCloudScore(mosaic)
mosaic_tk = scored_mosaic.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.4})

Mapdisplay(center,{'TOA mosaic':mosaic_tk},zoom_start=5)

`SENSOR_ID` is a property of individual images. When Earth Engine makes a mosaic of many images, it has to throw out individual image metadata, including the `SENSOR_ID` property. To cloud score a mosaic, Earth Engine looks for that property and can't find it, resulting in an error. Set the property manually to avoid that. The sensor IDs of Landsat 5, 7 and 8 are 'TM', 'ETM+' and 'OLI_TIRS', respectively.

### Simple composite

For creating simple cloud-free Landsat composites, Earth Engine provides the `ee.Algorithms.Landsat.simpleComposite()` method. This method selects a subset of scenes at each location, converts to TOA reflectance, applies the simple cloud score and takes the median of the least cloudy pixels. This example creates a simple composite using default parameters and compares it to a composite using custom parameters for the cloud score threshold and the percentile:

In [0]:
# Load a raw Landsat 5 ImageCollection for a single year.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1')\
               .filterDate('2010-01-01', '2010-12-31')

# Create a cloud-free composite with default parameters.
composite = ee.Algorithms.Landsat.simpleComposite(collection)

# Create a cloud-free composite with custom parameters for
# cloud score threshold and percentile.
customComposite = ee.Algorithms.Landsat.simpleComposite(
  collection=collection,
  percentile=75,
  cloudScoreRange=5
)

# Display the composites.
center=[37.7726,-122.3578]

composite_tk=composite.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 128})
customComposite_tk = customComposite.getMapId({'bands':['B4', 'B3', 'B2'], 'max':128})

Mapdisplay(center=center,
           dicc={'TOA composite':composite_tk,'Custom TOA composite':customComposite_tk},
           zoom_start=10)

Note that the input to the simple composite is a collection of raw imagery. Also note that by default, reflective band output is reflectance scaled to 8-bits and thermal band output is Kelvin minus 100, to fit in the 8-bit range. You can change this behavior by setting the `asFloat` parameter to true, to get un-scaled, un-shifted float output.

# 4. Sentinel-1 Algorithms

To create a homogeneous subset of Sentinel-1 data, it will usually be necessary to filter the collection using metadata properties. The common metadata fields used for filtering include these properties:

1. `transmitterReceiverPolarisation`:  ['VV'], ['HH'], ['VV', 'VH'], or ['HH', 'HV']

2. instrumentMode: 'IW' (Interferometric Wide Swath), 'EW' (Extra Wide Swath) or 'SM' (Strip Map). See [this reference](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/acquisition-modes) for details.

3. orbitProperties_pass: 'ASCENDING' or 'DESCENDING'.

4. resolution_meters: 10, 25 or 40

5. resolution: 'M' (medium) or 'H' (high). See [this reference](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/resolutions/level-1-ground-range-detected) for details.

For example:

In [0]:
# Load the Sentinel-1 ImageCollection.
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
              .filterDate('2015-01-01','2015-03-01')

# Filter by metadata properties.
# 1. Filter to get images with VV and VH dual polarization.
# 2. Filter to get images collected in interferometric wide swath mode.

vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
              .filter(ee.Filter.eq('instrumentMode', 'IW'))

# Filter to get images from different look angles.
vhAscending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vhDescending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# Create a composite from means at different polarizations and look angles.
composite = ee.Image.cat([
  vhAscending.select('VH').mean(),
  ee.ImageCollection(vhAscending.select('VV').merge(vhDescending.select('VV'))).mean(),
  vhDescending.select('VH').mean()
]).focal_median()

# Display as a composite of polarization and backscattering characteristics.
center=[37.7726,-122.3578]
composite_tk = composite.getMapId({'min': [-25, -20, -25], 'max': [0, 10, 0]})

Mapdisplay(center,{'composite':composite_tk},zoom_start=3)

### Sentinel-1 Preprocessing
Imagery in the Earth Engine `'COPERNICUS/S1_GRD'` Sentinel-1 `ImageCollection` is consists of Level-1 Ground Range Detected (GRD) scenes processed to backscatter coefficient (σ°) in decibels (dB). The backscatter coefficient represents target backscattering area (radar cross-section) per unit ground area. Because it can vary by several orders of magnitude, it is converted to dB as 10*log10σ°. It measures whether the radiated terrain scatters the incident microwave radiation preferentially away from the SAR sensor dB < 0) or towards the SAR sensor dB > 0). This scattering behavior depends on the physical characteristics of the terrain, primarily the geometry of the terrain elements and their electromagnetic characteristics.

Earth Engine uses the following preprocessing steps (as implemented by the [Sentinel-1 Toolbox](https://sentinel.esa.int/web/sentinel/toolboxes/sentinel-1)) to derive the backscatter coefficient in each pixel:

1. **Apply orbit file**
- Updates orbit metadata with a restituted orbit file.

2. **GRD border noise removal**
- Removes low intensity noise and invalid data on scene edges. (As of January 12, 2018)

3. **Thermal noise removal**
- Removes additive noise in sub-swaths to help reduce discontinuities between sub-swaths for scenes in multi-swath acquisition modes. (This operation cannot be applied to images produced before July 2015)

4. **Radiometric calibration**
- Computes backscatter intensity using sensor calibration parameters in the GRD metadata.

5. **Terrain correction (orthorectification)**
- Converts data from ground range geometry, which does not take terrain into account, to σ° using the [SRTM 30 meter DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003) or the [ASTER DEM](https://asterweb.jpl.nasa.gov/gdem.asp) for high latitudes (greater than 60° or less than -60°).

### Dataset Notes

- Radiometric Terrain Flattening is not being applied due to artifacts on mountain slopes.
- The unitless backscatter coefficient is converted to dB as described above.
- Sentinel-1 SLC data cannot currently be ingested, as Earth Engine does not support images with complex values due to inability to average them during pyramiding without losing phase information.


# 5. Resampling and Reducing Resolution

As noted in the [Projections](https://developers.google.com/earth-engine/projections) doc, Earth Engine performs nearest neighbor resampling by default during reprojection. You can change this behavior with the `resample()` or `reduceResolution()` methods. Specifically, when one of these methods is applied to an input image, any required reprojection of the input will be done using the indicated resampling or aggregation method.

## Resampling

`resample()` causes the indicated resampling method (`'bilinear'` or `'bicubic'`) to be used at the next reprojection. Since inputs are requested in the output projection, an implicit reprojection may happen before any other operation on the input. For this reason, call `resample()` directly on the input image. Consider the following simple example:

In [0]:
# Load a Landsat image over San Francisco, California, UAS.
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20160323')
# Force the next reprojection on this image to use bicubic resampling.
resampled = landsat.resample('bicubic')

# Set display and visualization parameters.
center = [37.6193,-122.37383]
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}

resampled_tk = resampled.getMapId(visParams)
landsat_tk = landsat.getMapId(visParams)

# Display the Landsat image using the default nearest neighbor and bicubic resampling.
# when reprojecting to Mercator for the Code Editor map.
Mapdisplay(center,{'raw':landsat_tk,'resampled':resampled_tk},zoom_start=15)

The order of operations for this code sample is diagrammed in Figure 1. Specifically, the implicit reprojection to the [maps mercator](http://epsg.io/3857) projection takes place with the resampling method specified on the input image.

<Figure>
<center>
  <img src="https://developers.google.com/earth-engine/images/Resample.png">
</center>
</Figure>

Figure 1. Flow chart of operations when `resample()` is called on the input image prior to display in the Code Editor. Curved lines indicate the flow of information to the reprojection: specifically, the output projection, scale and resampling method to use.

### Reduce Resolution

Suppose that instead of resampling during reprojection, your goal is to aggregate pixels to larger pixels in a different projection. This is useful when comparing image datasets at different scales, for example 30-meter pixels from a Landsat-based product to coarse pixels (higher scale) from a MODIS-based product. You can control this aggregation process with the `reduceResolution()` method. As with `resample()`, call `reduceResolution()` on the input, in order to affect the next reprojection of the image. The following example uses `reduceResolution()` to compare forest cover data at 30-meters resolution to a vegetation index at 500-meters resolution:

In [0]:
from pprint import pprint
# Load a MODIS EVI image.
modis = ee.Image(ee.ImageCollection('MODIS/006/MOD13A1').first())\
          .select('EVI')

# Display the EVI image near La Honda, California.
center = [37.5331, -122.3616]

# Get information about the MODIS projection.
modisProjection = modis.projection()

print('MODIS projection:')
pprint(modisProjection.getInfo())

# Load and display forest cover data at 30 meters resolution.
forest = ee.Image('UMD/hansen/global_forest_change_2015')\
           .select('treecover2000')

# Get the forest cover data at MODIS scale and projection.
# Force the next reprojection to aggregate instead of resampling.
# Request the data at the scale and projection of the MODIS image.
forestMean = forest.reduceResolution(reducer=ee.Reducer.mean(),
                                     maxPixels=1024)\
                   .reproject(crs=modisProjection)

# Display the aggregated, reprojected forest cover data.
forest_viz = {'max': 80}
modis_viz = {'min': 2000, 'max': 5000}

modis_tk = modis.getMapId(modis_viz)
forest_tk = forest.getMapId(forest_viz)
forestMean_tk = forestMean.getMapId(forest_viz)

Mapdisplay(center=center,
           dicc = {'forest cover 30 m':forest_tk,
                   'forest cover at MODIS scale':forestMean_tk,
                   'MODIS EVI':modis_tk},
           zoom_start=12)

MODIS projection:
{'crs': 'SR-ORG:6974',
 'transform': [463.312716528,
               0.0,
               -20015109.354,
               0.0,
               -463.312716527,
               10007554.677],
 'type': 'Projection'}


In this example, note that the output projection is explicitly set with [reproject()](https://developers.google.com/earth-engine/projections#reprojecting). During the reprojection to the MODIS sinusoidal projection, rather than resampling, the smaller pixels are aggregated with the specified reducer (`ee.Reducer.mean()` in the example). This sequence of operations is illustrated in Figure 2. Although this example uses `reproject()` to help visualize the effect of `reduceResolution()`, most scripts don't need to explicitly reproject; see the warning [here](https://developers.google.com/earth-engine/projections#reprojecting).

<center>
<img src='https://developers.google.com/earth-engine/images/ReduceResolution.png'>
</center>

Figure 2. Flow chart of operations when `reduceResolution()` is called on an input image prior to `reproject()`. Curved lines indicate the flow of information to the reprojection: specifically, the output projection, scale and pixel aggregation method to use.


Note that a second reprojection occurs (implicitly) to display the data on the Code Editor map. Visually inspect the results and observe the correspondence between the pixels from the MODIS layer and the forest cover data reprojected to MODIS scale and projection. In general, you should rarely need to explicitly reproject() in Earth Engine.

#### Pixel weights for ReduceResolution

The weights of pixels used during the reduceResolution() aggregation process are based on the overlap between the smaller pixels being aggregated and the larger pixels specified by the output projection. This is illustrated in Figure 3.

<center>
<img src="https://developers.google.com/earth-engine/images/ReduceResolution_weights.png">
</center>

Figure 3. Input pixels (black) and output pixel (blue) for `reduceResolution()`.

The default behavior is that input pixel weights are computed as the fraction of the output pixel area covered by the input pixel. In the diagram, the output pixel has area a, the weight of the input pixel with intersection area b is computed as b/a and the weight of the input pixel with intersection area c is computed as c/a. This behavior can result in unexpected results when using a reducer other than the mean reducer. For example, to compute forested area per pixel, use the mean reducer to compute the fraction of a pixel covered, then multiply by area (instead of computing areas in the smaller pixels then adding them up with the sum reducer):


In [0]:
# Compute forest area per MODIS pixel.
# 1.Force the next reprojection to aggregate instead of resampling.
# 2. The reduce resolution returns the fraction of the MODIS pixel
#    that's covered by 30 meter forest pixels.  Convert to area
#    after the reduceResolution() call.
# 3. Request the data at the scale and projection of the MODIS image.
forestArea = forest.gt(0)\
                   .reduceResolution(reducer=ee.Reducer.mean(),
                                         maxPixels=1024)\
                   .multiply(ee.Image.pixelArea())\
                   .reproject(crs=modisProjection)
forest_viz = {'max': 500 * 500}
forestArea_tk = forestArea.getMapId(forest_viz)

Mapdisplay(center=center,
           dicc={'forested area at MODIS scale':forestArea_tk})

<!--NAVIGATION-->
 < [Array](8_Array.ipynb) | [Contents](index.ipynb) |  [Asset Management](10_Import.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/9_SpecializedAlgorithms.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>